# Backend CV utilizing Keras CNN
This version is a backend version of Rock Paper Scissors. It uses a Keras CNN model to predict the hand gesture. The model was trained on a custom rock paper scissors dataset. Run this notebook to see a prediction of a hand gesture for rock paper scissors. 

In [1]:
import cv2
import HandTrackingDynamic as htd
from PIL import Image
import numpy as np
from keras import models

In [ ]:
import cv2
import mediapipe as mp
import math as math
class HandTrackingDynamic:
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.__mode__ = mode
        self.__maxHands__ = maxHands
        self.__detectionCon__ = detectionCon
        self.__trackCon__ = trackCon
        self.handsMp = mp.solutions.hands
        self.hands = self.handsMp.Hands()
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findFingers(self, frame, draw=True):
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(frame, handLms, self.handsMp.HAND_CONNECTIONS)

        return frame

    def findPosition(self, frame, handNo=0, handDraw=True, rectDraw=True):
        xList = []
        yList = []
        bbox = []
        self.lmsList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):

                h, w, c = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                self.lmsList.append([id, cx, cy])
                if handDraw:
                    cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
            # print("Hands Keypoint")
            # print(bbox)
            if rectDraw:
                cv2.rectangle(frame, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmsList, bbox

    def findFingerUp(self):
        fingers = []

        if self.lmsList[self.tipIds[0]][1] > self.lmsList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        for id in range(1, 5):
            if self.lmsList[self.tipIds[id]][2] < self.lmsList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers

    def findDistance(self, p1, p2, frame, draw=True, r=15, t=3):

        x1, y1 = self.lmsList[p1][1:]
        x2, y2 = self.lmsList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(frame, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(frame, (x2, y2), r, (255, 0, 0), cv2.FILLED)
            cv2.circle(frame, (cx, cy), r, (0, 0.255), cv2.FILLED)
        len = math.hypot(x2 - x1, y2 - y1)

        return len, frame, [x1, y1, x2, y2, cx, cy]


In [2]:
def verify_image_type(image):
    print("Image shape:", image.shape)
    print("Image data type:", image.dtype)


In [3]:
def resize():
    img = Image.open('handFrame.png')
    img = img.convert("RGB")
    img = img.resize((128, 128), Image.LANCZOS) 
    print("finished resize and recoloration")
    return np.array(img) / 255.0

In [4]:
model = models.load_model('../rpsModel.keras') # load in trained model
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
detector = htd.HandTrackingDynamic()
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

picture = False

In [5]:
while True:
    ret, frame = cap.read()
    orgFrame = frame.copy()
    frame = detector.findFingers(frame, draw=False)
    lmsList, bbox = detector.findPosition(frame, handDraw= False) #bbox should contain the bounding where the hand is
    cv2.imshow('RPS Identification', frame)
    if cv2.waitKey(1) == ord('p'):
        picture = True
    if cv2.waitKey(1) == ord('q'):
        break
    if lmsList and picture:
        xmin, ymin, xmax, ymax = bbox
        handFrame = orgFrame[ymin:ymax, xmin:xmax]
        cv2.imwrite('handFrame.png', handFrame)
        handArray = resize()
        prediction = model.predict(np.expand_dims(handArray, axis=0))
        print(prediction)
        PAPER, ROCK, SCISSORS = prediction[0][0], prediction[0][1], prediction[0][2]

        if ROCK > PAPER and ROCK > SCISSORS:
            print("Rock")
        elif PAPER > ROCK and PAPER > SCISSORS:
            print("Paper")
        else:
            print("Scissors")
        picture = False

cap.release()
cv2.destroyAllWindows()


finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
[[0.74545276 0.15088855 0.10365868]]
Paper
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[0.03253005 0.82130545 0.14616448]]
Rock
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[0.39032945 0.39082563 0.21884489]]
Rock
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[0.9921963  0.00395312 0.00385058]]
Paper
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[[0.88760376 0.0890481  0.02334821]]
Paper
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[0.04222639 0.12278486 0.8349888 ]]
Scissors
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
[[0.7861655  0.01975582 0.19407876]]
Paper
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
[[0.01947778 0.31180304 0.6687192 ]]
Scissors
finished resize and recoloration
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
[[0.00126614